In [1]:
# !pip install gdown

In [7]:
# скачиваем модель
!gdown https://drive.google.com/uc?id=1LDdPkkyvEK3ad6IvKjo9FgH84AKrGMDF

Downloading...
From: https://drive.google.com/uc?id=1LDdPkkyvEK3ad6IvKjo9FgH84AKrGMDF
To: /workspace/QA-for-Russian/classifier_epoch_2.pkl
1.43GB [02:18, 10.4MB/s]


In [3]:
import torch
from main_18_12 import Classifier, QADataset, train_model, main

In [18]:
# losses = main()

In [1]:
import numpy as np
import pandas as pd
import torch
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertForMaskedLM
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

import joblib

from preprocess_dataset import tokenize_text

device = ('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
data = pd.read_csv("sdsj2017_sberquad_with_spans(minus_30_examples).csv")
par_tokens = [i.split() for i in data.paragraph_tokens]
que_tokens = [tokenize_text(i) for i in data.question]
answer_spans = data.word_answer_span

word2index = {"[PAD]":0, "[CLS]":1, "[SEP]":2}

for sent in par_tokens:
    for token in sent:
        if token not in word2index:
            word2index[token] = len(word2index)

for que in que_tokens:
    for token in que:
        if token not in word2index:
            word2index[token] = len(word2index)

tokenizer = BertTokenizer.from_pretrained("lm", do_lower_case=False)

In [5]:
test = pd.read_csv("test.csv")

par_tokens_test = [i.split() for i in test.paragraph_tokens]
que_tokens_test = [tokenize_text(i) for i in test.question]
answer_spans_test = test.word_answer_span

test_data = QADataset(tokenizer=tokenizer,
                        paragraph_tokens=par_tokens_test,
                        question_tokens=que_tokens_test,
                        answer_spans=answer_spans_test,
                        word2index=word2index)

test_loader = DataLoader(test_data, batch_size=32, drop_last=True)

In [6]:
device = ('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# model = torch.load("classifier_epoch_2.pkl")
model = torch.load("classifier.pkl")
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=8e-5,
                                 weight_decay=0.01)

cuda


In [7]:
model

Classifier(
  (model): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dropout): Dropout(p

In [8]:
from test_and_evaluate import test_model, compute_f1

In [9]:
(all_f1_scores, 
 all_test_answers, 
 all_real_answers, 
 all_test_answer_spans, 
 all_real_answer_spans, 
 mean_test_losses) = test_model(model=model, 
                                test_loader=test_loader, 
                                criterion=criterion,
                               device=device)


Testing: test loss = 0.013 
[109, 179]
Evaluating: test accuracy = 0.000 
Evaluating: test F1-score = 0.062 



Testing: test loss = 0.105 
[139, 106]
Evaluating: test accuracy = 0.000 
Evaluating: test F1-score = 0.057 



Testing: test loss = 0.035 
[164, 214]
Evaluating: test accuracy = 0.000 
Evaluating: test F1-score = 0.049 


In [10]:
all_real_answers[0][:5]

['нуклеоид',
 'в 1026 году',
 'Лексические',
 'Bacillus fastidiosus',
 'в 1991 году']

In [11]:
all_test_answers[0][:5]

['слегка плейоморфен ( то есть способен изменять форму и размер в зависимости от условий ) и может принимать форму от овальной до блоковидной. Мимивирус является одним из крупнейших описанных вирусов и имеет икосаэдрический капсид диаметром 400 [UNK] 500 нм',
 'полководца Каппадокии или дука Месопотамии и стратега Каппадокии [ 13 ]',
 'обрести антоним. Таким образом, в контекстуальной антонимии антонимические отношения слов с прямым значением возможны, и тогда эти пары слов несут эмфатическую нагрузку и выполняют особую стилистическую функцию. [SEP] Какие единицы словарного состава языка оказываются тесно связанными не только на основании их ассоциативной связи по сходству',
 '',
 'волгоградского военного гарнизона, приняли участие в постсоветских войнах на Северном Кавказе и в 5 - дневной войне 2008 года в Южной Осетии']

In [12]:
all_real_answer_spans[0][:5]

['53:57', '225:228', '1:3', '305:310', '4:6']

In [13]:
all_test_answer_spans[0][:5]

['109:179', '190:218', '100:200', '108:81', '325:364']